<a href="https://colab.research.google.com/github/Choudhry18/Calendar_Schedular/blob/main/Calendar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install tesseract-ocr
!pip install pyTesseract
!pip install icalendar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached pytesseract-0.3.10-py3-none-any.whl (14 kB)


In [ ]:
import numpy as np
from PIL import Image
import pytesseract
import pytz
from icalendar import Calendar, Event
from datetime import datetime, time, timedelta
import dateutil
from dateutil.rrule import rrule, WEEKLY, MONTHLY, MO, TU, WE, TH, FR # For defining recurring rules
import re


In [ ]:
def imgToArray(imgLocation):
    img = Image.open(imgLocation)  # "C:/users/etuba/Desktop/ja.jpg")
    print(img.width, img.height)
    img = img.convert("L")
    return np.array(img)

def showImg(pixels):
    img = Image.fromarray(pixels,mode='L')
    img.show()

def time_to_TDelta(x):
  add = 0
  if (x[5] == 'P') and ((x[0]+x[1]) != '12'):
    add += 12
  if x[0] == '0':
    if x[3] == '0':
      return time(int(x[1])+add,int(x[4]))
    else:
      return time(int(x[1])+add,int(x[3]+x[4]))
  else:
    if x[3] == '0':
      return time(int(x[0]+x[1])+add,int(x[4]))
    else:
      return time(int(x[0]+x[1])+add,int(x[3]+x[4]))

In [14]:
    pixelsOrg = imgToArray("screen.png")
    img = Image.open("screen.png")
    #binarized the array
    th = 128
    pixels = np.copy(pixelsOrg)
    for i in range(len(pixels)):
        for j in range(pixels.shape[1]):
            if pixels[i][j] < th:
                pixels[i][j] = 0
            else:
                pixels[i][j] = 255
    count = 0
    line_h = []
    for i in range(len(pixels)):
        count = 0
        for j in range(pixels.shape[1]):
            if pixels[i][j] == 0:
                count+= 1
            if count>img.width-100:
                line_h.append(i)
                count = 0
                break
    #need to find a pair of y cordinates to calculate x cordinates for the 2 boxes
    #just making the line points worth one pixel
    count =0
    while count<len(line_h)-1:
        if line_h[count]+1 == line_h[count+1]:
            line_h.remove(line_h[count])
        else:
            count+=1
    first = line_h[0]
    second = line_h[1]
    line_v = []
    #finding the vertical line points
    for i in range(img.width):
        flag = True
        for j in range(first,second):
            if pixels[j][i] != 0:
                flag = False
                break
        if flag:
            line_v.append(i)

    count =0
    while count<len(line_v)-1:
        if line_v[count]+1 == line_v[count+1]:
            line_v.remove(line_v[count])
        else:
            count+=1



1600 94


In [24]:
#getting the list of courses and their description using OCR
courses = []
for i in range(len(line_h)-1):
  courses.append(re.sub("\n"," ",pytesseract.image_to_string(pixels[line_h[i]:line_h[i+1], line_v[0]:line_v[1]],config='--psm 6')))
descriptions = []
for i in range(len(line_h)-1):
  descriptions.append(re.sub("\n"," ",pytesseract.image_to_string(pixels[line_h[i]:line_h[i+1], line_v[2]:line_v[3]],config='--psm 6')))


In [25]:
#Removing the course number and other gibresh from the course name
print(courses)
for i in range(len(courses)):
  course = courses[i]
  new = ''
  flag = False
  for j in [*course]:
    if flag:
      new += j
    if j == ")":
      flag = True
  courses[i] = new

print(courses)


['RELI-3444-1 (86656) Islam in North America \x0c', 'BIOL-1398-4 [86686] The Microo al Mothersh p_ \x0c', '/ARTH-3459-1 (86608) Modemism in the Visual Arts \x0c', 'ART-3450-1 (866023 Photograghy II \x0c', '‘CSCI-3198-1 (86823) Technical Interview Prep. \x0c']
[' Islam in North America \x0c', '', ' Modemism in the Visual Arts \x0c', '', ' Technical Interview Prep. \x0c']


In [ ]:
pattern = re.compile(r'\b(?:Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)\b')
date_P = r'\d{1,2}/\d{1,2}/\d{4}'
time_P = r'\b\d{1,2}:\d{2}(?:AM|PM)\b'
r_pattern = r'Room \b\d{3}\b'
del_list = []
days_mapping = {
    "Monday": MO,
    "Tuesday": TU,
    "Wednesday": WE,
    "Thursday": TH,
    "Friday": FR,
}
dates = []
rooms = []
times = []
days = []
# using pattern matching to get info from descriptions
for i in range(len(descriptions)):
  desc = descriptions[i]
  dates.append(re.findall(date_P, desc))
  rooms.append(re.findall(r_pattern, desc))
  times.append(re.findall(time_P, desc))
  index = 0
  n = []

  #This groups the days that appear next to each other together in a list. Like
  # Monday,Wednesday would be together.
  matches = pattern.finditer(desc)
  for j in matches:
    start_position = j.start()
    end_position = j.end()
    if index+15 < start_position:
      days.append(n)
      n = [j.group(0)]
    else:
      n.append(j.group(0))
    index = j.end()
  days.append(n)


#Extracting the buildings from the description by finding the occurences of the
#room nums and adding the comma seperated part of desc before it.
buildings = []
for i in range(len(descriptions)):
  n = []
  count = -1
  desc_list = descriptions[i].split(", ")
  for room in rooms[i]:
    for part in range(len(desc_list)):
      if len(desc_list[part]) >= len(room):
        if desc_list[part][0:len(room)] == room:
          if count >= 0:
            if desc_list[part-1] != n[count]:
              n.append(desc_list[part-1])
              count += 1
          else:
            n.append(desc_list[part-1])
            count += 1
  buildings.append(n)

#This removes the repitions in days
for i in range(1,len(days)):
  if (days[i] == days[i-1]):
    del_list.append(i)

neg = 0
for i in del_list:
  del days[i-neg]
  neg += 1
count = -1
ndays = []
print(days)
for i in range(len(days)):
  if days[i] == []:
    ndays.append([])
    count += 1
    continue
  for j in range(len(days[i])):
    days[i][j] = days_mapping[days[i][j]]
  ndays[count].append(tuple(days[i]))
print(ndays)

ntimes = []
for i in times:
  n = []
  count = 0
  for j in range(0, len(i), 2):
    if j+1<len(i):
      n.append((time_to_TDelta(i[j]),time_to_TDelta(i[j+1])))
  ntimes.append(n)
print(ntimes)


[[], ['Monday', 'Wednesday'], [], ['Monday', 'Wednesday', 'Friday'], [], ['Monday'], [], ['Tuesday', 'Thursday'], [], ['Thursday']]
[[(MO, WE)], [(MO, WE, FR)], [(MO,)], [(TU, TH)], [(TH,)]]
[[(datetime.time(13, 6), datetime.time(14, 20))], [(datetime.time(10, 30), datetime.time(11, 20))], [(datetime.time(14, 30), datetime.time(17, 10))], [(datetime.time(9, 55), datetime.time(11, 45))], [(datetime.time(12, 45), datetime.time(14, 0))]]


In [ ]:
print(buildings)
print(rooms)
print(ndays[1][0][0])

[['Cir for Scisnces & Innovation'], [], ['Dicke/Smith Building'], [], ['Ctr for Sciences & Innovation']]
[['Room 204'], [], ['Room 341'], ['Room 206'], ['Room 483']]
MO


In [ ]:
cal = Calendar()
start_date = datetime(2024, 1, 10)
end_date = datetime(2024, 4, 30)
for i in range(len(ndays)):
  nevent = len(ndays[i])
  for j in range(nevent):
    rday = ndays[i][j]
    timeT = ntimes[i][j]
    time_diff = timedelta(hours=timeT[1].hour, minutes=timeT[1].minute) - timedelta(hours=timeT[0].hour, minutes=timeT[0].minute)
    rrule_params = {
    'freq': WEEKLY,
    'byweekday': rday,
    'dtstart': datetime.combine(start_date, timeT[0]),
    'until': datetime.combine(end_date, timeT[0]),
     }
    event_schedule = rrule(**rrule_params)
    for event_time in event_schedule:
      event_end = event_time + time_diff
      event = Event()
      event.add('summary', courses[i])
      event.add('dtstart', event_time)
      event.add('dtend', event_end)
      cal.add_component(event)

with open('recurring_event.ics', 'wb') as f:
    f.write(cal.to_ical())

print("iCalendar file 'recurring_event.ics' created.")


iCalendar file 'recurring_event.ics' created.


In [32]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN

# Load the image
image_path = "screen.png"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(image, (5, 5), 0)

# Perform Canny edge detection
edges = cv2.Canny(blurred, 50, 150, apertureSize=3)

# Use Hough Line Transform to detect lines
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)

# Separate the lines into horizontal and vertical lines
horizontal_lines = []
vertical_lines = []

if lines is not None:
    for line in lines:
        for x1, y1, x2, y2 in line:
            if abs(y1 - y2) < 10:  # Horizontal line threshold
                horizontal_lines.append((x1, y1, x2, y2))
            elif abs(x1 - x2) < 10:  # Vertical line threshold
                vertical_lines.append((x1, y1, x2, y2))

# Convert lines to points for clustering
horizontal_points = np.array([(x1, y1, x2, y2) for x1, y1, x2, y2 in horizontal_lines])
vertical_points = np.array([(x1, y1, x2, y2) for x1, y1, x2, y2 in vertical_lines])

# Perform DBSCAN clustering if points are available
clustered_horizontal_lines = []
clustered_vertical_lines = []

def cluster_lines(points):
    if points.shape[0] == 0:
        return []
    dbscan = DBSCAN(eps=10, min_samples=2).fit(points[:, :2])  # Cluster based on start points
    clusters = []
    for cluster_id in set(dbscan.labels_):
        if cluster_id != -1:  # Ignore noise points
            cluster_points = points[dbscan.labels_ == cluster_id]
            x1, y1 = np.min(cluster_points[:, :2], axis=0)
            x2, y2 = np.max(cluster_points[:, 2:], axis=0)
            clusters.append((x1, y1, x2, y2))
    return clusters

clustered_horizontal_lines = cluster_lines(horizontal_points)
clustered_vertical_lines = cluster_lines(vertical_points)

# Draw the clustered lines on the image
output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

for x1, y1, x2, y2 in clustered_horizontal_lines:
    cv2.line(output_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

for x1, y1, x2, y2 in clustered_vertical_lines:
    cv2.line(output_image, (x1, y1), (x2, y2), (255, 0, 0), 2)

# Save or display the result
cv2.imwrite("output.png", output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

